<a href="https://colab.research.google.com/github/jhonyzuim/Python_para_Data_Science/blob/master/Aula_6_Jhony.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 Configurando o Spark no Python**

Etapa responsável por realizar o download e configuração do Spark

In [1]:
#Instacao e configuracao do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#Descompatacao do Apache Spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#Instalacao do Findspark
!pip install -q findspark

**2 Configurando ambiente**

Eatapa responsável por configurar o ambiente do Spark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

**3 Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 Validando a Configuração e versão do Spark**

Etapa de verificação

In [4]:
import pyspark
print(pyspark.__version__)

3.0.1


**5 Download dos Dados Portal da Transparencia**

Etapa responsável por realizar o Download das informações de Gastos por meio de cartão de pagamento providas através da API: http://www.portaltransparencia.gov.br/api-de-dados/cartoes?pagina=1

In [8]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: dc48f972261fe427cdfa8b8da7d5c768' 'http://www.portaltransparencia.gov.br/api-de-dados/cnep?pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43346    0 43346    0     0  42832      0 --:--:--  0:00:01 --:--:-- 42832


**6 Carregando os dados no Spark**

Etapa paga a carga de dados no Spark.

In [9]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre as Viagens
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

**7 Explorando os dados que foram obtidos**

Etapa de exploração dos dados.

In [10]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- abrangenciaDefinidaDecisaoJudicial: string (nullable = true)
 |-- dataFimSancao: string (nullable = true)
 |-- dataInicioSancao: string (nullable = true)
 |-- dataOrigemInformacao: string (nullable = true)
 |-- dataPublicacaoSancao: string (nullable = true)
 |-- dataReferencia: string (nullable = true)
 |-- dataTransitadoJulgado: string (nullable = true)
 |-- detalhamentoPublicacao: string (nullable = true)
 |-- fonteSancao: struct (nullable = true)
 |    |-- enderecoContato: string (nullable = true)
 |    |-- nomeExibicao: string (nullable = true)
 |    |-- telefoneContato: string (nullable = true)
 |-- id: long (nullable = true)
 |-- informacoesAdicionaisDoOrgaoSancionador: string (nullable = true)
 |-- legislacao: struct (nullable = true)
 |    |-- descricaoFundamentacaoLegal: string (nullable = true)
 |    |-- fundamentacaoLegal: string (nullable = true)
 |-- linkPublicacao: string (nullable = true)
 |-- numeroProcesso: string (nullable = true)
 |-- orgaoSancionador: stru

**8 Tratamento e transformação de Dados**



In [12]:
dfPrepared = df.selectExpr("dataInicioSancao","pessoa","tipoSancao","valorMulta")
dfPrepared.show()

+----------------+--------------------+--------------------+------------+
|dataInicioSancao|              pessoa|          tipoSancao|  valorMulta|
+----------------+--------------------+--------------------+------------+
|      07/05/2020|[[Comércio vareji...|[Multa - Lei 12.8...|   14.346,00|
|      07/05/2020|[[Comércio vareji...|[Multa - Lei 12.8...|  264.852,00|
|      05/10/2020|[[Carga e descarg...|[Publicação Extra...|        0,00|
|      15/05/2020|[[Comércio vareji...|[Multa - Lei 12.8...|   70.295,79|
|      20/10/2020|[[Sem informação,...|[Multa - Lei 12.8...|  221.920,45|
|      11/03/2020|[[Fabricação de p...|[Multa - Lei 12.8...|   51.410,99|
|      20/08/2020|[[Comércio vareji...|[Multa - Lei 12.8...|  168.219,45|
|      20/08/2020|[[Comércio vareji...|[Publicação Extra...|        0,00|
|      14/08/2019|[[Atividades técn...|[Publicação Extra...|        0,00|
|      14/08/2019|[[Atividades técn...|[Multa - Lei 12.8...|   12.281,37|
|      29/08/2019|[[Serviços de eng...

**9 Analise das informações e resultados obtidos**

Etapa onde são realizados as analises utilizando linguagem SQL para por exemplo descobrir quem gastou mais esse ano

In [13]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [18]:
topTravels = spark.sql("select count(*),pessoa from dfPrepared group by 2 order by 1 desc")
topTravels.show()

+--------+--------------------+
|count(1)|              pessoa|
+--------+--------------------+
|       2|[[Sem informação,...|
|       2|[[Atividades técn...|
|       2|[[Comércio vareji...|
|       2|[[Comércio vareji...|
|       1|[[Fabricação de p...|
|       1|[[Fabricação de p...|
|       1|[[Serviços de eng...|
|       1|[[Comércio vareji...|
|       1|[[Carga e descarg...|
|       1|[[Serviços de eng...|
|       1|[[Sem informação,...|
+--------+--------------------+

